## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from gensim.models import KeyedVectors

## Carregando os dados

In [ ]:
artigo_treino = pd.read_csv('Dados/treino.csv')
artigo_teste = pd.read_csv('Dados/teste.csv')


## Visualizando os dados

In [ ]:
# Visualizando os dados de treino.
artigo_treino.sample(5)

In [ ]:
# Visualizando os dados de teste.
artigo_teste.sample(5)

## Lendo cbow.txt

In [ ]:
# Lendo o arquivo de vetores pré-treinados.
with open (r'.\Dados\cbow_s300.txt\cbow_s300.txt', 'r', encoding='utf-8') as f:
    for linha in range(10):
        print(next(f))

In [ ]:
modelo = KeyedVectors.load_word2vec_format(r'.\Dados\cbow_s300.txt\cbow_s300.txt')

## Vetorização de texto

### Tokenização

In [ ]:
# Criando uma função para tokenizar o texto.
def tokenizador(texto):
    # deixando o texto em minúsculo
    texto = texto.lower()

    # criando uma lista vazia para armazenar os tokens.
    lista_alfanumerico = []

    # para cada token no texto, se o token não estiver na lista de pontuações, adicione-o à lista de tokens.
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerico.append(token_valido)      

    return lista_alfanumerico

In [ ]:
# Criando uma função para vetorizar o texto.
def combinacao_de_vetores_por_soma(palavras_numeros):
    # definindo um vetor de tamanho 300 com valores 0.
    vetor_resultante = np.zeros(300)

    # para cada palavra-numero no texto, soma o vetor da palavra ao vetor_resultante.
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)    
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector('unknown')
    return vetor_resultante

## Vetorizando os dados

In [ ]:
# Criando uma função para vetorizar os dados de treino e teste.
def matriz_vetores(textos):
    matriz = np.zeros((len(textos), 300))

    for i in range(len(textos)):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)

    return matriz

# Vetorizando os dados de treino e teste.
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)